In [4]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
import lightgbm as lgb


def to_hdf(dataset,name):
    dataset_train=dataset[pd.isna(dataset['target'])==False].copy()
    dataset_test=dataset[pd.isna(dataset['target'])].copy()
    dataset_train.to_hdf('dataset/'+name+'.h5', key='TRAIN')
    dataset_test.to_hdf('dataset/'+name+'.h5', key='PREDICT')
def read_hdf(name):
    dataset_train = pd.read_hdf('dataset/'+name+'.h5', key='TRAIN')
    dataset_test = pd.read_hdf('dataset/'+name+'.h5', key='PREDICT')
    return pd.concat([dataset_train,dataset_test])
dataset_train = pd.read_csv('dataset/LOAN_TRAIN_SET.csv',header=0,index_col ='cust_no')
dataset_test = pd.read_csv('dataset/LOAN_PREDICT_SET_01.csv',header=0,index_col ='cust_no')
dataset_train.to_hdf('dataset/1_原数据转存.h5', key='TRAIN')
dataset_test.to_hdf('dataset/1_原数据转存.h5', key='PREDICT')

fill_0=['cus_age','cus_mbank_lcnt','cus_active_ind','pass_date','interval_mon_count','interval_date_1stc','interval_date_1stc12','cus_hisloan_cnt','crcd_mavg_balind']
fill_avg=['cus_region_move_ind','cus_reloan_cap_ind','cus_asset_bal','crcd_max_quota','crcd_lmon_due_amount','crcd_due_amount','crcd_mavg_balance']
fill_f1=['crcd_points','a_mmax_tranbal','a_mavg_tranbal','b_mmax_tranbal','b_mavg_tranbal','c_mmax_tranbal','c_mavg_tranbal','d_mmax_tranbal','d_mavg_tranbal','e_mmax_tranbal','e_mavg_tranbal','f_mmax_tranbal','f_mavg_tranbal','g_mmax_tranbal','g_mavg_tranbal','h_mmax_tranbal','h_mavg_tranbal','i_mmax_tranbal','i_mavg_tranbal','j_mmax_tranbal','j_mavg_tranbal','k_mmax_tranbal','k_mavg_tranbal','m_mmax_tranbal','m_mavg_tranbal','n_mavg_trancount','n_mavg_tranbal']
fill_f1_div=['a_mdivision_tranbal','b_mdivision_tranbal','c_mdivision_tranbal','d_mdivision_tranbal','e_mdivision_tranbal','f_mdivision_tranbal','g_mdivision_tranbal','h_mdivision_tranbal','i_mdivision_tranbal','j_mdivision_tranbal','k_mdivision_tranbal','m_mdivision_tranbal','n_mdivision_tranbal','crcd_due_div_crcd_max_quota','crcd_due_div_crcd_lmon_due','a_div_cus_asset_bal','b_div_cus_asset_bal','c_div_cus_asset_bal','d_div_cus_asset_bal','e_div_cus_asset_bal','f_div_cus_asset_bal','g_div_cus_asset_bal','h_div_cus_asset_bal','i_div_cus_asset_bal','j_div_cus_asset_bal','k_div_cus_asset_bal','m_div_cus_asset_bal','n_div_cus_asset_bal']

confinuous_feature=fill_0+fill_avg+fill_f1+fill_f1_div
cago_feature=['crcd_is_overdue','cus_sex','crcd_is_pc','crcd_is_sup','cus_type','crcd_is_gec','crcd_bachange_cnt','cus_intvtime_trans','cus_edu','cus_marr_stat','cus_occu','cus_oact_pla','cus_os_dist','crcd_is_gc','crcd_bill_mcnt']

dataset=read_hdf('1_原数据转存') 
dataset[fill_0]=dataset[fill_0].fillna(0)
dataset[fill_f1]=dataset[fill_f1].fillna(0)

for c in fill_avg:
    dataset[c]=dataset[c].fillna(dataset[c].mean())


dataset[cago_feature]=dataset[cago_feature].fillna(-999)
for c in cago_feature:
    dataset[c]=LabelEncoder().fit_transform(dataset[c])
    
# dataset['index']=range(len(dataset))
dataset['a_mdivision_tranbal']=dataset['a_mmax_tranbal']/dataset['a_mavg_tranbal']
dataset['b_mdivision_tranbal']=dataset['b_mmax_tranbal']/dataset['b_mavg_tranbal']
dataset['c_mdivision_tranbal']=dataset['c_mmax_tranbal']/dataset['c_mavg_tranbal']
dataset['d_mdivision_tranbal']=dataset['d_mmax_tranbal']/dataset['d_mavg_tranbal']
dataset['e_mdivision_tranbal']=dataset['e_mmax_tranbal']/dataset['e_mavg_tranbal']
dataset['f_mdivision_tranbal']=dataset['f_mmax_tranbal']/dataset['f_mavg_tranbal']
dataset['g_mdivision_tranbal']=dataset['g_mmax_tranbal']/dataset['g_mavg_tranbal']
dataset['h_mdivision_tranbal']=dataset['h_mmax_tranbal']/dataset['h_mavg_tranbal']
dataset['i_mdivision_tranbal']=dataset['i_mmax_tranbal']/dataset['i_mavg_tranbal']
dataset['j_mdivision_tranbal']=dataset['j_mmax_tranbal']/dataset['j_mavg_tranbal']
dataset['k_mdivision_tranbal']=dataset['k_mmax_tranbal']/dataset['k_mavg_tranbal']
dataset['m_mdivision_tranbal']=dataset['m_mmax_tranbal']/dataset['m_mavg_tranbal']
dataset['n_mdivision_tranbal']=dataset['n_mavg_trancount']/dataset['n_mavg_tranbal']
dataset['crcd_due_div_crcd_max_quota']=dataset['crcd_max_quota']/dataset['crcd_due_amount']
dataset['crcd_due_div_crcd_lmon_due']=dataset['crcd_lmon_due_amount']/dataset['crcd_due_amount']
dataset['a_div_cus_asset_bal']=dataset['a_mavg_tranbal']/dataset['cus_asset_bal']
dataset['b_div_cus_asset_bal']=dataset['b_mavg_tranbal']/dataset['cus_asset_bal']
dataset['c_div_cus_asset_bal']=dataset['c_mavg_tranbal']/dataset['cus_asset_bal']
dataset['d_div_cus_asset_bal']=dataset['d_mavg_tranbal']/dataset['cus_asset_bal']
dataset['e_div_cus_asset_bal']=dataset['e_mavg_tranbal']/dataset['cus_asset_bal']
dataset['f_div_cus_asset_bal']=dataset['f_mavg_tranbal']/dataset['cus_asset_bal']
dataset['g_div_cus_asset_bal']=dataset['g_mavg_tranbal']/dataset['cus_asset_bal']
dataset['h_div_cus_asset_bal']=dataset['h_mavg_tranbal']/dataset['cus_asset_bal']
dataset['i_div_cus_asset_bal']=dataset['i_mavg_tranbal']/dataset['cus_asset_bal']
dataset['j_div_cus_asset_bal']=dataset['j_mavg_tranbal']/dataset['cus_asset_bal']
dataset['k_div_cus_asset_bal']=dataset['k_mavg_tranbal']/dataset['cus_asset_bal']
dataset['m_div_cus_asset_bal']=dataset['m_mavg_tranbal']/dataset['cus_asset_bal']
dataset['n_div_cus_asset_bal']=dataset['n_mavg_tranbal']/dataset['cus_asset_bal']
dataset[fill_f1_div]=dataset[fill_f1_div].fillna(-1)
# for c in confinuous_feature:
#     dataset.loc[dataset[c]==np.inf,c]=-1
# dataset[confinuous_feature] = preprocessing.scale(dataset[confinuous_feature])

to_hdf(dataset,'1_2_填充')

/home/jiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
def get_sample(dataset,target_0=0.33,target_1=0.33,seed=None):
    dataset=dataset.copy()
    x0=dataset[dataset['target']==0]
    x1=dataset[dataset['target']==1]
    x0_selected=x0.sample(frac=target_0,random_state=seed)
    x1_selected=x1.sample(frac=target_1,random_state=seed)
    dataset=pd.concat([x0_selected,x1_selected]).sample(frac=1,random_state=seed)
    y_selected=dataset['target']
    x_selected=dataset.drop(columns =['target'])
    return x_selected,y_selected
dataset_train=pd.read_hdf('dataset/1_2_填充.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_2_填充.h5', key='PREDICT')
feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc', 'crcd_is_sup', 'cus_type', 'crcd_is_gec', 'crcd_bachange_cnt',
             'cus_intvtime_trans', 'cus_edu', 'cus_marr_stat', 'cus_oact_pla', 'crcd_is_gc', 'crcd_bill_mcnt',
             'cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'k_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'crcd_due_div_crcd_lmon_due', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal'
        ]
cago_feature='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14'
feature_train=feature+['target']
feature_test=feature
dataset_train=dataset_train[feature_train]
dataset_test=dataset_test[feature_test]
dataset_train_x,dataset_train_y=get_sample(dataset_train[feature_train],target_0=1,target_1=1,seed=None)
X_train,X_test,Y_train,Y_test=train_test_split(dataset_train_x,dataset_train_y,test_size=0.2)

params={'num_leaves':12,
    'min_data_in_leaf':100,
    'objective':'binary',
    'max_depth':-1,
    'learning_rate':0.02,
    'boosting':'gbdt',
    'bagging_freq':8,
    'bagging_fraction':0.8,
    'feature_fraction':0.4,
    'bagging_seed':8,
    'reg_alpha':8,
    'reg_lambda':33,
    'random_state':42,
    'metric':'auc',
    'verbosity':-1,
    'subsample':1,
    'min_gain_to_split':0.01,
    'min_child_weight':10,
    'num_threads':-1,
    'is_unbalance': True, 'categorical_feature': cago_feature}
lgb_train = lgb.Dataset(X_train,Y_train)
lgb_eval = lgb.Dataset(X_test,Y_test,reference=lgb_train)

model=lgb.train(params,lgb_train,num_boost_round=4000,valid_sets=lgb_eval,early_stopping_rounds=200)

[1]	valid_0's auc: 0.609105
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.630845
[3]	valid_0's auc: 0.641551
[4]	valid_0's auc: 0.678458
[5]	valid_0's auc: 0.676215
[6]	valid_0's auc: 0.681375
[7]	valid_0's auc: 0.680483
[8]	valid_0's auc: 0.689353
[9]	valid_0's auc: 0.691102
[10]	valid_0's auc: 0.692208
[11]	valid_0's auc: 0.691695
[12]	valid_0's auc: 0.691751
[13]	valid_0's auc: 0.694217
[14]	valid_0's auc: 0.694492
[15]	valid_0's auc: 0.695248
[16]	valid_0's auc: 0.695647
[17]	valid_0's auc: 0.696024
[18]	valid_0's auc: 0.697354
[19]	valid_0's auc: 0.697306
[20]	valid_0's auc: 0.697566
[21]	valid_0's auc: 0.698425
[22]	valid_0's auc: 0.698629
[23]	valid_0's auc: 0.698372
[24]	valid_0's auc: 0.69871
[25]	valid_0's auc: 0.69897
[26]	valid_0's auc: 0.698428
[27]	valid_0's auc: 0.699058
[28]	valid_0's auc: 0.69953
[29]	valid_0's auc: 0.700062
[30]	valid_0's auc: 0.69991
[31]	valid_0's auc: 0.700002
[32]	valid_0's auc: 0.700508
[33]	valid_0's auc: 0.7

[323]	valid_0's auc: 0.716617
[324]	valid_0's auc: 0.716649
[325]	valid_0's auc: 0.716736
[326]	valid_0's auc: 0.716848
[327]	valid_0's auc: 0.716969
[328]	valid_0's auc: 0.717018
[329]	valid_0's auc: 0.717053
[330]	valid_0's auc: 0.717003
[331]	valid_0's auc: 0.717036
[332]	valid_0's auc: 0.717123
[333]	valid_0's auc: 0.7171
[334]	valid_0's auc: 0.717031
[335]	valid_0's auc: 0.716978
[336]	valid_0's auc: 0.716883
[337]	valid_0's auc: 0.716911
[338]	valid_0's auc: 0.716935
[339]	valid_0's auc: 0.717011
[340]	valid_0's auc: 0.717011
[341]	valid_0's auc: 0.717019
[342]	valid_0's auc: 0.71704
[343]	valid_0's auc: 0.717057
[344]	valid_0's auc: 0.717051
[345]	valid_0's auc: 0.717164
[346]	valid_0's auc: 0.717188
[347]	valid_0's auc: 0.717232
[348]	valid_0's auc: 0.717241
[349]	valid_0's auc: 0.717305
[350]	valid_0's auc: 0.717365
[351]	valid_0's auc: 0.717293
[352]	valid_0's auc: 0.717297
[353]	valid_0's auc: 0.717267
[354]	valid_0's auc: 0.717302
[355]	valid_0's auc: 0.717331
[356]	valid_0

[606]	valid_0's auc: 0.720728
[607]	valid_0's auc: 0.720784
[608]	valid_0's auc: 0.720776
[609]	valid_0's auc: 0.720797
[610]	valid_0's auc: 0.72075
[611]	valid_0's auc: 0.720764
[612]	valid_0's auc: 0.720758
[613]	valid_0's auc: 0.720733
[614]	valid_0's auc: 0.720659
[615]	valid_0's auc: 0.72062
[616]	valid_0's auc: 0.720593
[617]	valid_0's auc: 0.720601
[618]	valid_0's auc: 0.720623
[619]	valid_0's auc: 0.72067
[620]	valid_0's auc: 0.720663
[621]	valid_0's auc: 0.72065
[622]	valid_0's auc: 0.720641
[623]	valid_0's auc: 0.720635
[624]	valid_0's auc: 0.720638
[625]	valid_0's auc: 0.720601
[626]	valid_0's auc: 0.72058
[627]	valid_0's auc: 0.720589
[628]	valid_0's auc: 0.720577
[629]	valid_0's auc: 0.720602
[630]	valid_0's auc: 0.720613
[631]	valid_0's auc: 0.720624
[632]	valid_0's auc: 0.720601
[633]	valid_0's auc: 0.720596
[634]	valid_0's auc: 0.720543
[635]	valid_0's auc: 0.720577
[636]	valid_0's auc: 0.720547
[637]	valid_0's auc: 0.720551
[638]	valid_0's auc: 0.720575
[639]	valid_0's

[906]	valid_0's auc: 0.721105
[907]	valid_0's auc: 0.721068
[908]	valid_0's auc: 0.721065
[909]	valid_0's auc: 0.72105
[910]	valid_0's auc: 0.721063
[911]	valid_0's auc: 0.721048
[912]	valid_0's auc: 0.721063
[913]	valid_0's auc: 0.72108
[914]	valid_0's auc: 0.721092
[915]	valid_0's auc: 0.72107
[916]	valid_0's auc: 0.721083
[917]	valid_0's auc: 0.721124
[918]	valid_0's auc: 0.721121
[919]	valid_0's auc: 0.721134
[920]	valid_0's auc: 0.721167
[921]	valid_0's auc: 0.721166
[922]	valid_0's auc: 0.721181
[923]	valid_0's auc: 0.72116
[924]	valid_0's auc: 0.721184
[925]	valid_0's auc: 0.721177
[926]	valid_0's auc: 0.721212
[927]	valid_0's auc: 0.721203
[928]	valid_0's auc: 0.72119
[929]	valid_0's auc: 0.721187
[930]	valid_0's auc: 0.721176
[931]	valid_0's auc: 0.721189
[932]	valid_0's auc: 0.721163
[933]	valid_0's auc: 0.72116
[934]	valid_0's auc: 0.721163
[935]	valid_0's auc: 0.721141
[936]	valid_0's auc: 0.721142
[937]	valid_0's auc: 0.721151
[938]	valid_0's auc: 0.721167
[939]	valid_0's 

[1205]	valid_0's auc: 0.721864
[1206]	valid_0's auc: 0.721882
[1207]	valid_0's auc: 0.721899
[1208]	valid_0's auc: 0.721919
[1209]	valid_0's auc: 0.721934
[1210]	valid_0's auc: 0.72193
[1211]	valid_0's auc: 0.721922
[1212]	valid_0's auc: 0.721916
[1213]	valid_0's auc: 0.721948
[1214]	valid_0's auc: 0.721939
[1215]	valid_0's auc: 0.721929
[1216]	valid_0's auc: 0.721932
[1217]	valid_0's auc: 0.721953
[1218]	valid_0's auc: 0.721954
[1219]	valid_0's auc: 0.721996
[1220]	valid_0's auc: 0.721968
[1221]	valid_0's auc: 0.721981
[1222]	valid_0's auc: 0.721995
[1223]	valid_0's auc: 0.722045
[1224]	valid_0's auc: 0.722033
[1225]	valid_0's auc: 0.722037
[1226]	valid_0's auc: 0.72204
[1227]	valid_0's auc: 0.722048
[1228]	valid_0's auc: 0.722036
[1229]	valid_0's auc: 0.722003
[1230]	valid_0's auc: 0.722016
[1231]	valid_0's auc: 0.721996
[1232]	valid_0's auc: 0.721984
[1233]	valid_0's auc: 0.722006
[1234]	valid_0's auc: 0.722009
[1235]	valid_0's auc: 0.721998
[1236]	valid_0's auc: 0.721938
[1237]	val